In [107]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

In [108]:
sentence = ['people watching campusx', 'campusx watching campusx', 'people writing comment', 'campusx writing comment']
value  = [1,1,0,0]
dic = {'sentences':sentence,
       'values':value}
data = pd.DataFrame(dic)
data2 = data.copy()

In [109]:
def tokenize(text):
    words = word_tokenize(text)
    return words

def lemmatizer(token_list):
    wnl = WordNetLemmatizer()
    return ([wnl.lemmatize(word,'v') for word in token_list])

data['sentences'] = data['sentences'].apply(tokenize)
data['sentences'] = data['sentences'].apply(lemmatizer)

In [110]:
data

,sentences,values
0,"[people, watch, campusx]",1
1,"[campusx, watch, campusx]",1
2,"[people, write, comment]",0
3,"[campusx, write, comment]",0


In [111]:
corpus = []
for i in range(data.shape[0]):
    corpus = corpus + data['sentences'][i]
corpus

['people',
 'watch',
 'campusx',
 'campusx',
 'watch',
 'campusx',
 'people',
 'write',
 'comment',
 'campusx',
 'write',
 'comment']

In [112]:
vocabulary = list(np.unique(corpus))
vocabulary

['campusx', 'comment', 'people', 'watch', 'write']

In [113]:
data['sentences'][2]

['people', 'write', 'comment']

In [114]:
def idf(text):
    count_of_term = 0.000001
    number_of_docs = data.shape[0]
        
    for i in range(number_of_docs):
        if text in data['sentences'][i]:
            count_of_term+=1
    idf = (np.log(number_of_docs/count_of_term))+1
    return idf
    
def tf(text,doc):
    number_of_terms = len(doc)
    count_of_term = doc.count(text)
        
    tf = count_of_term/number_of_terms
    return tf

In [115]:
def tf_idf(doc):
    vector = []
    for text in vocabulary:
        tfs = tf(text,doc)
        idfs = idf(text)
        tfidf = np.round((tfs*idfs),3)
        vector.append(tfidf)
    return vector

In [116]:
idf('campusx')

1.2876817391185031

In [117]:
tf_idf(data['sentences'][0])

[0.429, 0.0, 0.564, 0.564, 0.0]

In [118]:
data['sentences'] = data['sentences'].apply(tf_idf)

In [119]:
data

,sentences,values
0,"[0.429, 0.0, 0.564, 0.564, 0.0]",1
1,"[0.858, 0.0, 0.0, 0.564, 0.0]",1
2,"[0.0, 0.564, 0.564, 0.0, 0.564]",0
3,"[0.429, 0.564, 0.0, 0.0, 0.564]",0


# TF-IDF Vectorizer

In [120]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [121]:
data2

,sentences,values
0,people watching campusx,1
1,campusx watching campusx,1
2,people writing comment,0
3,campusx writing comment,0


In [122]:
tidf = TfidfVectorizer()
fitter = tidf.fit_transform(data2['sentences']).toarray()

In [123]:
fitter

array([[0.49681612, 0.        , 0.61366674, 0.61366674, 0.        ],
       [0.8508161 , 0.        , 0.        , 0.52546357, 0.        ],
       [0.        , 0.57735027, 0.57735027, 0.        , 0.57735027],
       [0.49681612, 0.61366674, 0.        , 0.        , 0.61366674]])

In [124]:
print(tidf.idf_)
print(tidf.get_feature_names_out())

[1.22314355 1.51082562 1.51082562 1.51082562 1.51082562]
['campusx' 'comment' 'people' 'watching' 'writing']
